# Reservoir operation modelling
In this Notebook we will see how simulation models and optimisation algorithms can be used to assist the operation of a water reservoir system.

<left><img src="../../util/images/Dam1.gif" width = "800px"><left>
    
We consider a simple illustrative system where a reservoir is operated to supply water to a domestic consumption node, while ensuring a minimum environmental flow in the downstream river (also called “environmental compensation flow”) and maintaining the water level in the reservoir within prescribed limits. We use a mathematical model to link all the key variables that represent the reservoir dynamics (inflow, storage and outflows) and use model simulation/optimisation to determine the reservoir release scheduling that will best meet the water demand over a coming period of time, given the predicted (or assumed) scenario of future inflows.
<left> <img src="../../util/images/system_representation_IO0.png" width = "600px"><left>

## Import libraries
Before getting started, let's import some libraries that will be used throughout the Notebook:

In [1]:
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import numpy as np
import ipywidgets as widgets
from Modules.Interactive_release_schedule import Interactive_release_single,Interactive_release_double, Interactive_Pareto_front
warnings.filterwarnings('ignore') # to ignore warning messages

## The reservoir model

The mathematical model of the reservoir essentially consists of a water balance equation, where the storage (***s***) at a future time step (for example, at the beginning of the next week) is predicted from the storage at the current time (the beginning of the this week) by adding and subtracting the inflows and outflows that will occur during the temporal interval ahead:

$s(t+1) = s(t) + I(t) – E(t) – env(t) - spill(t) – Qreg(t)$   

Where

***s(t)*** = reservoir storage at time-step t, in Vol (for example: ML)

***I(t)*** = reservoir inflows in the interval [t,t+1], in Vol/time (for example: ML/week). This is usually provided by a flow forecasting system or assumed by looking, for example, at historical inflow records for the relevant time of year

***E(t)*** = evaporation from the reservoir surface area in the interval [t,t+1], in Vol/time (for example: ML/week). This is calculated internally to the model, by multipling the evaporation rate for unit surface area (which depends on air temperature) by the reservoir surface area (which is derived from the storage S given that the reservoir geometry is known)

***env(t)*** = environmental compensation flow in the interval [t,t+1], in Vol/time (for example: ML/week). This is usually set to the value that was agreed upon with the environemtal regulator ([Learn more about the rational behind environmental flows](https://www.youtube.com/watch?v=cbUrrYq9BmU))

***spill(t)*** = outflow through spillways (if any) in the interval [t,t+1], in Vol/time (for example: ML/week). This is calculated internally to the model, and is equal to the excess volume with respect to the maximum reservoir capacity (so most of the time ***spill(t)*** is equal to 0 as the maximum capacity is not reached, but it occasionally is >0 so that the capacity is never exceeded)

***Qreg(t)*** = regulated reservoir release for water supply in the interval [t,t+1], in Vol/time (for example: ML/week). This is a completely free variable that the reservoir operator will need to specify
<left> <img src="../../util/images/system_representation_IO1.png" width = "600px"><left>

#### Implementation of the reservoir simulation function
Here we define a function that implements the reservoir simulation, that is, iteratively apply the mass balance equation and reconstruct the temporal evolution of the reservoir variables over the simulation period

In [2]:
def syst_sim(N,I,e,d,S0,Smax,env_min,Qreg):

    # Declare output variables

    S = [0]*(N+1) # reservoir storage in ML

    spill = [0]*N # spillage in ML

    env = [env_min]*N # environmental compensation flow
    
    S[0] = S0 # initial storage

    for t in range(N): # Loop for each time-step (week)

        # If at week t the inflow (I) is lower than the minimum environmental compensation (env_min), 
        # then the environmental compensation (env) = inflow (I)  
        if env_min >= I[t] :
            env[t] = I[t]
        # If the minimum environmental compensation is higher than the water resource available (S + I - E)
        # then the environmental compensation is equal to the higher value between 0 and the resource available
        if env_min >= S[t] + I[t] - e[t]:
            env[t] = max(0,S[t] + I[t] - e[t]) 
        # If the demand is higher than the water resource available (S + I - E - env)
        # then the release for water supply is equal to the higher value between 0 and the resource available            
        if d[t] >= S[t] + I[t] - e[t] - env[t]:
            Qreg[t] = min(Qreg[t],max(0,S[t] + I[t] - e[t] - env[t]))
        # The spillage is equal to the higher value between 0 and the resource available exceeding the reservoir capacity
        spill[t] = max(0,S[t] + I[t] - Qreg[t] - env[t] - e[t] - Smax)
        # The final storage (initial storage in the next step) is equal to the storage + inflow - outflows
        S[t+1] = S[t] + I[t] - Qreg[t] - env[t]- e[t] - spill[t]
        
    return S,env,spill,Qreg

#### Definition of inflow and demand scenarios
Let's assume we want to look at the next 8 weeks the number of weeks (so ***N=8***), and assume we have forecasts of inflows and demand for this period.

In [3]:
N = 8 # (weeks) length of the simulation period

I_fore = np.array([15,17,19,11,9,4,3,8]) # (ML/week) time series of inflow forecasts
T_fore = np.array([13,13,17,18,20,22,25,26]) # (degC) time series of temperature forecasts
d_fore = np.array([15]*N)*T_fore/15 # (ML/week) time series of demand forecasts, estimated as a function of temperature
# (this is onviously a very simplified approach, and one could use a much more sophisticated demand model)

Plot the inflow and demand forecasts:

In [4]:
# Axis characterisitcs
x_sc_1 = LinearScale();y_sc_1 = LinearScale(min=0,max=35)
x_ax_1 = Axis(label='week', scale=x_sc_1);y_ax_1 = Axis(label='ML/week', scale=y_sc_1, orientation='vertical')
# Bar plot
inflow_plot = plt.bar(np.arange(1,N+1),I_fore,colors=['blue'],stroke = 'lightgray',scales={'x': x_sc_1, 'y': y_sc_1},
                      labels = ['inflow'], display_legend = True)
#Figure characteristics
fig_1a = plt.Figure(marks = [inflow_plot],title = 'Inflow forecast for the next 8 weeks', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '1000px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_1a])

In [5]:
# Bar plot (we use the same axis as the weekly inflows figure)
demand_plot   = plt.bar(np.arange(1,N+1),d_fore,colors=['gray'],stroke = 'lightgray',opacities = [1]*N, 
                        labels = ['demand'], display_legend = True, 
                    stroke_width = 1,scales={'x': x_sc_1, 'y': y_sc_1})
#Figure characteristics
fig_1b = plt.Figure(marks = [demand_plot],title = 'Demand forecast for the next 8 weeks', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '1000px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 1})
widgets.VBox([fig_1b])

#### Definition of other input parameters
Let's define other variables that are needed for the reservoir system simulation, such as the reservoir storage capacity, the environmental compensation flow, etc.

In [6]:
### Constraints ###
s_max = 150 #  (ML) Maximum storage (=reservoir capacity)
s_min = 0 # (ML) Minimum storage (set to zero for now)
env_min = 2 # (ML/week)   # Environmental compensation flow

### Initial conditions ###
s_0 = 80 # (ML) # Storage volume at the beginning of the simulation period
e_fore = T_fore*0.1 # (ML/week) Time series of evaporation from the reservoir (this is a very simplified 
# approach and could be replaced by a more realistic estimation approach)

## Determining the release scheduling by trial and error (manual optimisation)
Here we want to use the reservoir model to assist the reservoir operator in determining the best scheduling of regulated reservoir releases (Qreg) in response to a certain scenario of inflows. The goal is to minimise the deficit with respect to a prescribed water demand, that is, to minimise the objective function:

$$TSD = \sum_{t=1}^{N} [ \ max( \ 0, \ d(t)-Qreg(t) \ ) \ ]^2 $$

where N is the length of the simulation period that we are considering, and d(t) is the water demand for each time-interval in that period, and TSD stands for Total Squared Deficit. Notice that the function $max(0,...)$ enables us to only count the difference between demand d and release u when this is positive, that is, when the release u is smaller than the demand d, and a water shortage is indeed produced. Also, the squaring is a 'mathematical trick' to make sure that larger deficit amounts are given more weight than smaller ones. This translates the fact that small deficit amounts are easier to mitigate and hence more acceptable, while larger ones can cause disproportionately severe impacts and should be avoided as much as possible.

#### Determining the optimal release scheduling via interactive visualisation

Use the slider to set the release amount for each week in the simulation period, and in doing so try to minimise the Total Squared Deficit. 

In [7]:
# Interactive release scheduling
fig_2a,fig_2b,release1,release2,release3,release4,release5,release6,release7,release8 = Interactive_release_single(
    N,I_fore,e_fore,d_fore,s_0,s_max,env_min, demand_plot)
Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [release1,release2,release3,release4,release5,release6,release7,release8],layout=Box_layout),fig_2b,fig_2a],layout=Box_layout)

**Comment**: clearly it is not possible to fully meet the demand at all times in the simulation period. For example if we fully cover the demand for the first 7 weeks, we drawdown the reservoir to a point that we are forced to dramatically reduce the release in the last week, causing a very severe deficit. A more effective approach is to cause smaller deficits across all time steps, that is, tolerate some small deficits even when in principle we may fully cover the demand, in order to prevent more severe deficits in the later period. This type of approach is called ***hedging*** (see for example [You and Cai 2008](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2006WR005481)).

Btw, the minimum TSD value that can be achieved is **49, try to beat it!**

## From single to multi-objective
Now let's assume that, besides minimising supply deficits, we are also interested in minimising the chances that the reservoir level go below a minimum threshold. This could be, for example, because the quality of the water deteriorates when levels are low, requiring more costly treatment. We measure how well this criterion is satisfied by the following objective function:

$$MSV = \sum_{t=1}^{N} max ( \ rc - S(t) , \ 0)$$

where, again, N is the length of the simulation period, S is the reservoir storage, and rc is the minimum reservoir storage threshold that should preferably not be transpassed (MSV stands for Minimum Storage Violation). 

For our case, let's set this threshold to 30 ML:

In [8]:
# Minimum storage threshold
ms = np.array([30]*(N+1)) # in ML

Now use the slider to set the release amount for each week in the simulation period, in a way that jointly minimise the Total Squared Deficit and the Minimum Storage Violation. 

In [9]:
fig_3a,fig_3b,release1,release2,release3,release4,release5,release6,release7,release8 = Interactive_release_double(
    N,I_fore,e_fore,d_fore,s_0,s_max,ms,env_min, demand_plot)
Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [release1,release2,release3,release4,release5,release6,release7,release8],layout=Box_layout),fig_3b,fig_3a],layout=Box_layout)

***Comment*** It is possible to find a release scheduling that produce no violation of the minimum storage threshold, although it will produce some supply deficit - the record is **305, can you beat it?**. However, one could also allow some violation of the storage threshold in order to reduce the deficits. The two objectives are conflicting: improving on one of them implies doing worse on the other.

### References 

Deb K. et al (2002) A fast and elitist multiobjective genetic algorithm: NSGA-II, IEEE Transactions on Evolutionary Computation, 6(2), 182-197, doi:10.1109/4235.996017.

You J.Y. and Cai X. (2008) Hedging rule for reservoir operations: 1. A theoretical analysis, Water Resour. Res., 44, W01415, doi:10.1029/2006WR005481.

#### Let's go to the next section!: [3. Calibration and evaluation of a rainfall-runoff model](3.%20Calibration%20and%20evaluation%20of%20a%20rainfall-runoff%20model.ipynb)